# K-gram Index

### Made By Yuliia Hetman

In [1]:
%%time
from multiprocessing import Pool
import re, os
from string import digits


docIDs = list(range(0, 18))


def isUkrainian(word):
    ukrainian = [96, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1100, 1102, 1103, 1108, 1110, 1111, 1169, 8217]
    for l in word:
        if ord(l) not in ukrainian:
            return False
    return True

def check_vocabulary(vocabulary):
        return ['$' + k + '$' for k in set(vocabulary) if isUkrainian(k) == True]
        
def delete_sings_digits(content):
    punc = '''!()-'”№[]{};:"\,“«»<>./?@#$%—…^&*_~|–abcdefghijklmnoqprstuvwxyz'''
    content = "".join([ele for ele in content.lower() if ele not in punc])
    table = str.maketrans('', '', digits)
    content = content.translate(table)
    return content


def create_vocabulary(content):    
    raw_content = delete_sings_digits(content)
    vocab = [i.lower() for i in raw_content.split() if i != '']
    return vocab

def read_book(book):
    dirname='../books/'
    with open(dirname + book, 'r', encoding='utf-8') as file:
        text = create_vocabulary(file.read())
    return text

        
def get_vocabulary(dirname='../books/'):
    books = os.listdir(dirname)
    vocabulary = list()
    dictionary = dict()
    memory = 0
    with Pool(8) as p:
        vocabul = p.map(read_book, books)
    for i, book in enumerate(books):
        dictionary[book.replace('.txt', '')] = vocabul[i]
        vocabulary += vocabul[i]
        memory += os.stat(dirname + book).st_size
    return memory, vocabulary, dictionary


memory, vocabulary, dictionary = get_vocabulary()
vocabulary = check_vocabulary(vocabulary)
print(f'Total number of words : {len(vocabulary)}')
print(f'Total number of tokens : {len(set(vocabulary))}')
print('\n', vocabulary[:3],'\n')

Total number of words : 93799
Total number of tokens : 93799

 ['$відвернуло$', '$полковницький$', '$вібравши$'] 

CPU times: user 993 ms, sys: 290 ms, total: 1.28 s
Wall time: 1.42 s


In [2]:
%%time


def split_word_into_k_grams(k, word):
    return [(word[i:i + k], word) for i in range(len(word) - k + 1)]

def parallelize_splitting(word):
    k_grams = list()
    K = [2, 3, 4]
    for k in K:
        k_grams += split_word_into_k_grams(k, word)
    return k_grams


class K_Grams():
    def __init__(self, vocabulary):
        self.vocabulary = vocabulary
        self.final_grams = dict()
        self.grams = self.split_all_words()
        self.k_grams = list()
        self.max_k = 4
    
    def split_all_words(self):
        with Pool(10) as p:
            self.k_grams = p.map(parallelize_splitting, self.vocabulary)
        for word in self.k_grams:
            for (gram, w) in word:
                if (gram in self.final_grams):
                    self.final_grams[gram] += [w]
                else:
                    self.final_grams[gram] = [w]
        return self.final_grams
    

    def find(self, string):
        if len(string) > self.max_k:
            return self.grams[string[:max_k]]
        else:
            return self.grams[string]


CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 31.7 µs


### Example of how grams are stored

In [3]:
def AND(word1, word2):
    return list(set(word1) & set(word2))

In [13]:

def processing_boolean_req(grams, string, k=3):
    s = '$' + string + '$'
    jocker = string.find('*')
    if jocker == -1:
        return grams.find(string[:k])
    parts = string.split('*')
    l1 = grams.find(parts[0])
    for i in range(1, len(parts)):
        l1 = AND(l1, grams.find(parts[i][:k]))
    return l1


def compare_particles(term, parts, temp=0):
    for i in range(len(parts)):
        ind = term.find(parts[i])
        if temp < ind:
            temp = ind
            continue
        else:
            return False
    return True


def post_filtering(results, template, k=3):
    satisfactory = list()
    start = '$' + template[:k - 1]
    end = template[1 - k :] + '$'
    parts = template.split('*')
    for word in results:
        if word[:k] == start and word[-k :] == end and compare_particles(word, parts) == True:
            satisfactory.append(word)
        else:
            continue
    return satisfactory


def beautiful_head_print(words):
    for word in words[:5]:
        print(word[1:-1])
    print('\n')

In [14]:
%%time

template = 'по*ки'
grams = K_Grams(vocabulary)
result_bool_request = processing_boolean_req(grams, template)
checked_results = post_filtering(result_bool_request, template)
print(checked_results)

['$покатівськи$', '$понімецьки$', '$покритки$', '$потихеньки$', '$посусідськи$', '$політки$', '$похапки$', '$полумиски$', '$поволеньки$', '$поміщики$', '$подорожники$', '$позапорозьки$', '$посмішки$', '$повітки$', '$похристиянськи$', '$поприятельськи$', '$подруженьки$', '$пописьменськи$', '$порошки$', '$посторонки$', '$починки$', '$полковники$', '$познавецьки$', '$полянки$', '$подушки$', '$полонинки$', '$поцілунки$', '$політики$', '$помолодецьки$', '$полички$', '$полюдськи$', '$полядськи$', '$подножки$', '$подалисятаки$', '$потоки$', '$посиденьки$', '$поголоски$', '$постаросвітськи$', '$польки$', '$поговірки$', '$початки$', '$поразки$', '$поазіатськи$', '$покоролівськи$', '$поклики$', '$помосковськи$', '$поварварськи$', '$понаськи$', '$подалеки$', '$поворозки$', '$попанцьки$', '$повозки$', '$попаски$', '$покрадьки$', '$потайники$', '$половики$', '$повіки$', '$покоївки$', '$потемки$', '$поминки$', '$попанськи$', '$помилки$', '$порицарськи$', '$подарунки$', '$побиванки$', '$поясники$', '

In [15]:
%%time
beautiful_head_print(checked_results)

покатівськи
понімецьки
покритки
потихеньки
посусідськи


CPU times: user 534 µs, sys: 0 ns, total: 534 µs
Wall time: 320 µs
